# Capstone Project - Battle of the Neighborhoods - Rome Versus Paris
### Thomas KLEIN

In this notebook I will perform the final Capstone Project, named Battle of the Neighborhoods, for the Applied Data Science Capstone Coursera course. 

## 1. Introduction/Business Problem

#### 1.1 Background
Rome and Paris are two major capitals of important European countries, respectively France and Italy. Both capitals are known to be major tourist destinations worldwide as well places famous for their cuisine and gastronomy. As shown by the numbers below, they have a comparable size and population as well as are ranked 2nd and 3rd on worldatlas.com in cities attracting the highest number of tourists in Europe in 2017 (1st is London).

|                                           | Rome          | Paris        | Source                                |
| ------------------------------------------|:-------------:| :-----------:|---------------------------------------|
| **Official population (city limits)**     | 2,857,321     | 2,140,526    |*(Wikipedia)*                          |
| **Tourists (international arrivals 2017)**| 9.5 million   | 14.26 million|*(worldatlas.com)*                     |
| **Number of restaurants**                 | 13,721 (2017) |44,896 (2017) |*(worldcitiescultureforum.com)*        |

As a Frenchman and a foodie, I feel engaged in the existing rivalry between these two cities and decided to analyze and compare them further in order to explore how similar or different they are as well as conclude on which city has the most diverse restaurant scene and the best ratings on Foursquare.
Although Foursquare ratings are only partial, subjective and dependent on many variables, we can consider that since Foursquare has an international user base and that both cities are visited by important numbers of tourists, the users and the ratings are comparable.

#### 1.2 Target group
While there are many different popular cuisines all over the world, the two main ones that often come back in comparison in discussions in western Europe are the French and the Italian ones regarding which one is the most popular. Foodies and gastronomy tourists from different countries would be interested in the results of a study comparing the ratings of both cities. 
For most also, Paris is considered as more multicultural than Rome would be. An outcome of the present study will be to verify which city has the most diverse food scene, which can also be of interest of both local inhabitants and international gastronomy tourists looking to decide on which city to visit.

#### 1.3 Problem setting
The objective of this study and this workbook will be to answer 3 questions:
1. How similar are the neighborhoods of Paris and Rome and if they are dissimilar, what distinguishes them?
2. Which of the two cities has the most diverse food scene?
3. Which of the two cities has the best ratings (both in total and for their national cuisine)?  

In the next section we will cover the data needed to answer these questions.

## 2. Data

In order to solve the problems set above, we will need different sources of data.

##### A breakdown of the two cities into "Neighborhoods"
The first type of data will be way to divide and separate the different areas in Rome and Paris in order to answer the first question which is to compare the neighborhoods of both cities and find similarities or differences. To do so the following lists and sources of information will be used:
- [the Wikipedia page describing the 'Administrative subdivision of Rome' into 15 municipi](https://it.wikipedia.org/wiki/Municipi_di_Roma)
- [the Wikipedia page describing the 'Arrondissements of Paris' into 20 arrondissements](https://en.wikipedia.org/wiki/Arrondissements_of_Paris)

##### GPS coordinates to be used with the Foursquare API and database
The second type of data are GPS coordinates to be able to locate each neighborhood and search venues around using the Foursquare API. To do so we can use the geolocator package but in case we do not obtain correct information we can use the following websites:
- [This website](https://www.coordinatesmarker.com/results/x5CIL-z14eDjgYvrFSXe) provides the GPS coordinates of the all 'municipi' of the city of Rome. A CSV file containing has been created and stored [here](https://github.com/ThomasKlein90/Coursera_Capstone/blob/master/coordinatesmarker.com_ROMA.csv).
- [The same website](https://www.coordinatesmarker.com/results/YJtUk9HYiG8XQX0k28q9) has been used to get the coordinates of the different 'arrondissements' of Paris. A CSV file containing has been created and stored [here](https://github.com/ThomasKlein90/Coursera_Capstone/blob/master/coordinatesmarker.com_PARIS.csv).

##### The Foursquare database - list of venues
The third source of information and data is the Foursquare database and API which contains venues in both cities. Accessing the database using the GPS coordinates listed in the files mentioned above we can characterize the neighborhoods by obtaining the categories of the venues. Further exploring the venues and their categories we can isolate the types of restaurants and their numbers to answer the second question.

##### The Foursquare database - restaurants ratings
Finally, still using the Foursquare database and isolating the restaurants in both cities we can extract their ratings to be able to compare them between Rome and Paris.

## 3. Methods

#### 3.1. Obtaining the list of neighborhoods in Rome and Paris
In order to first obtain the lists of 15 municipi in Rome and 20 arrondissements in Paris we are going to use the beautiful soup package in order to scrape the two tables we want from the Wikipedia pages mentioned in the section above.

In [1]:
### We start by importing the different librairies and packages we need
import pandas as pd
import numpy as np
import requests #We import requests library to ping website for html code

!pip install beautifulsoup4 #We install the beautifulsoup4 package
!pip install lxml #We also install a HTML parser

     |████████████████████████████████| 102kB 1.8MB/s ta 0:00:01
     |████████████████████████████████| 5.8MB 1.9MB/s eta 0:00:01


In [2]:
#We store the html codes of both Wikipedia pages using requests.get(url).text
rome_url=requests.get('https://it.wikipedia.org/wiki/Municipi_di_Roma').text
paris_url=requests.get('https://en.wikipedia.org/wiki/Arrondissements_of_Paris').text

In [3]:
#We begin reading the html of the websites using a Beautiful Soup object and the Prettify() function
from bs4 import BeautifulSoup
rome_soup = BeautifulSoup(rome_url, 'lxml')
paris_soup = BeautifulSoup(paris_url, 'lxml')

#We look for the object 'wikitable sortable' corresponding to the table we want to scrape
Paris_table = paris_soup.find('table', class_='wikitable sortable')
Rome_table = rome_soup.find('table', class_='wikitable sortable')

In [4]:
#The data we want to capture is contained within the signs 'th' and 'td'. We start by reading only this bit of the table.
Paris_data=Paris_table.find_all(['th','td'][1::1])

#We initialize our 2 lists for neighborhoods and their names
Paris_Neighborhoods=[]
Paris_Names=[]

#We create a while loop to look through the table and assign the text in the first cell to neighborhoods and in the second to names.
#Then we iterate i to i+8 (number of columns in the original table) to look in the next line.
i=0
while i in range(0,len(Paris_table.find_all('td'))):
    Neighborhood=Paris_data[i].text
    Name=Paris_data[i+1].text
    Paris_Neighborhoods.append(Neighborhood)
    Paris_Names.append(Name)
    i=i+8

#We create a Dataframe to import each of our 2 created lists as each one our columns.
Paris_df=pd.DataFrame()
Paris_df['Neighborhood']=Paris_Neighborhoods
Paris_df['Name']=Paris_Names
Paris_df.head(5)

,Neighborhood,Name
0,1st (Ie) R,Louvre
1,2nd (IIe) R,Bourse
2,3rd (IIIe) R,Temple
3,4th (IVe) R,Hôtel-de-Ville
4,5th (Ve) L,Panthéon


In [5]:
#We do the same to create the Rome table
#The data we want to capture is contained within the signs 'th' and 'td'. We start by reading only this bit of the table.
Rome_data=Rome_table.find_all(['th','td'][1::1])

#We initialize our 1 list for neighbourhoods
Rome_Neighborhoods=[]

#We create a while loop to look through the table and assign the text in the first cell to neighborhoods.
#Then we iterate i to i+7 (number of columns in the original table) to look in the next line.
i=0
while i in range(0,len(Rome_table.find_all('td'))):
    Neighborhood=Rome_data[i].text
    Neighborhood=Neighborhood.strip()  #we remove the \n at the end of each Neighborhood name
    Rome_Neighborhoods.append(Neighborhood)
    i=i+7

#We create a Dataframe to import our list.
Rome_df=pd.DataFrame()
Rome_df['Neighborhood']=Rome_Neighborhoods
Rome_df=Rome_df.join(Rome_df['Neighborhood'].str.split(' ', 1, expand=True)) #we split the indicator of the Municipi and the name of the municipi into two different columns
Rome_df=Rome_df.drop(columns=['Neighborhood']).rename(columns={0:'Neighborhood', 1:'Neighborhood_Name'})
Rome_df=Rome_df.drop(index=[15,16]) #we drop the last two rows which do not contain any neighborhoods
Rome_df.head(5)

,Neighborhood,Neighborhood_Name
0,I,Centro Storico
1,II,Parioli/Nomentano
2,III,Monte Sacro
3,IV,Tiburtina
4,V,Prenestino/Centocelle


#### 3.2. Merging it with the coordinates lists
As mentioned in the Data section, we have already obtained and stored csv files of the coordinates of the neighborhoods of Rome and Paris. Now we need to import them, reorganize them, verify them, and finally merge them with the dataframes created in 3.1. 

##### Rome

In [6]:
# We read the csv file stored on the Github repository containing the coordinated of the Municipi
Rome_coordinates_df=pd.read_csv('https://raw.githubusercontent.com/ThomasKlein90/Coursera_Capstone/master/coordinatesmarker.com_ROMA.csv')
Rome_coordinates_df.head(5)

,Name,Latitude,Longitude
0,Municipio Roma I,41.902915,12.482536
1,Municipio Roma II,41.917486,12.499319
2,Municipio Roma III,42.001921,12.555902
3,Municipio Roma IV,41.932972,12.597107
4,Municipio Roma V,41.894312,12.573932


In order to merge both tables we need the Municipi to have the same names in both tables Rome_df and Rome_coordinates_df. We will settle on the naming "Municipio Roma xxx" as the final name to identify the Municipi. An extra column "Neighborhood_Name" will contain the more descriptive name of the Neighboorhood.

In [7]:
#We create a new Dataframe containing the start of the neighborhood name "Municipio Roma "
Municipi = pd.Series([])
for i in range(len(Rome_df['Neighborhood'])):
    Municipi[i]='Municipio Roma '
Rome_df['Neighborhood']=Municipi+Rome_df['Neighborhood']

#We merge both tables based on the Neighborhood column in Rome_df and Name column in Rome_coordinates_df
Rome_df=Rome_df.join(Rome_coordinates_df.set_index('Name'), on='Neighborhood')
Rome_df.head(5)

,Neighborhood,Neighborhood_Name,Latitude,Longitude
0,Municipio Roma I,Centro Storico,41.902915,12.482536
1,Municipio Roma II,Parioli/Nomentano,41.917486,12.499319
2,Municipio Roma III,Monte Sacro,42.001921,12.555902
3,Municipio Roma IV,Tiburtina,41.932972,12.597107
4,Municipio Roma V,Prenestino/Centocelle,41.894312,12.573932


##### Paris
We do the same for Paris. We will choose to name the neighborhoods "Paris arrondissement xxx" and another column will indicate a more descriptive name for the neighborhoods.

In [8]:
# We read the csv file stored on the Github repository containing the coordinated of the Municipi
Paris_coordinates_df=pd.read_csv('https://raw.githubusercontent.com/ThomasKlein90/Coursera_Capstone/master/coordinatesmarker.com_PARIS.csv')
Paris_coordinates_df.head(5)

,Name,Latitude,Longitude
0,Paris arrondissement XX,48.861827,2.401073
1,Paris arrondissement XIX,48.879275,2.392504
2,Paris arrondissement XVIII,48.892126,2.348178
3,Paris arrondissement XVII,48.887439,2.306523
4,Paris arrondissement XVI,48.874541,2.291396


This does not look right for arrondissements V, X, XI which have completely different coordinates from the other ones. Other sources of data gives us the following information so we can correct the coordinates for the erroneous neighborhoods. We also reverse the order of the table.

| Name                    | Latitude     | Longitude       |
|:-----------------------:| :-----------:|-----------------|
|Paris 5e                 | 48.8454189   |2.3525815263157  |
|Paris 10e                | 48.87600805  |2.3604450364092  |
|Paris arrondissement 11e | 48.8561815   |2.3709781        |

In [9]:
Paris_coordinates_df=Paris_coordinates_df.reindex(index=Paris_coordinates_df.index[::-1]).reset_index(drop=True)
Paris_coordinates_df.iloc[4, 1] = 48.8454189
Paris_coordinates_df.iloc[4, 2] = 2.3525815263157
Paris_coordinates_df.iloc[9, 1] = 48.87600805
Paris_coordinates_df.iloc[9, 2] = 2.3604450364092
Paris_coordinates_df.iloc[10, 1] = 48.8561815
Paris_coordinates_df.iloc[10, 2] = 2.3709781
Paris_coordinates_df.head(5)

,Name,Latitude,Longitude
0,Paris arrondissement I,48.834188,2.331955
1,Paris arrondissement II,48.867053,2.363482
2,Paris arrondissement III,48.863583,2.287129
3,Paris arrondissement IV,48.857323,2.362473
4,Paris arrondissement V,48.845419,2.352582


We treat the dataframe obtained in section 3.1 to extract the arrondissement number in the list and change it to "Paris arrondissement xxx"

In [10]:
Paris_df2=Paris_df.join(Paris_df['Neighborhood'].str.split(' ', 1, expand=True)) #we split the indicator of the Municipi and the name of the municipi into two different columns
Paris_df2=Paris_df2.rename(columns={0:'A', 1:'B'})
Paris_df3=Paris_df.join(Paris_df2['B'].str.split(' ', 1, expand=True).rename(columns={0:'A', 1:'B'}))
Paris_df4=Paris_df.join(Paris_df3['A'].str.replace(r'[^(]*\(|\)[^)]*', ''))
Paris_df5=Paris_df.join(Paris_df4['A'].str.replace('e','')).drop(columns=['Neighborhood'])
Paris_df5=Paris_df5.rename(columns={'Name':'Neighborhood_Name','A':'Neighborhood'})

Arrondissements = pd.Series([])
for i in range(len(Paris_df5['Neighborhood_Name'])):
    Arrondissements[i]='Paris arrondissement '
Paris_df5['Neighborhood']=Arrondissements+Paris_df5['Neighborhood']
Paris_df=Paris_df5[['Neighborhood','Neighborhood_Name']]
Paris_df.head(5)

,Neighborhood,Neighborhood_Name
0,Paris arrondissement I,Louvre
1,Paris arrondissement II,Bourse
2,Paris arrondissement III,Temple
3,Paris arrondissement IV,Hôtel-de-Ville
4,Paris arrondissement V,Panthéon


Finally we can merge the Paris_coordinates_df table with the Paris_df table to add the coordinates.

In [11]:
#We merge both tables based on the Neighborhood column in Paris_df and Name column in Paris_coordinates_df
Paris_df=Paris_df.join(Paris_coordinates_df.set_index('Name'), on='Neighborhood')
Paris_df.head(5)

,Neighborhood,Neighborhood_Name,Latitude,Longitude
0,Paris arrondissement I,Louvre,48.834188,2.331955
1,Paris arrondissement II,Bourse,48.867053,2.363482
2,Paris arrondissement III,Temple,48.863583,2.287129
3,Paris arrondissement IV,Hôtel-de-Ville,48.857323,2.362473
4,Paris arrondissement V,Panthéon,48.845419,2.352582


#### 3.3 Visualizing the Neighborhoods of both cities on maps
We can first create Folium maps to visualize the neighborhood of both cities on maps to make sure we have correct coordinates.

In [23]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [24]:
import folium

#We initialize the Folium map with Paris's general coordinates
Paris_lat=48.864716
Paris_long=2.349014
map_Paris = folium.Map(location=[Paris_lat, Paris_long], zoom_start=13)

#We start adding to the map the centers of the different neighborhoods listed in the dataframe we created previously
for lat, lng, neighborhood, name in zip(Paris_df['Latitude'],Paris_df['Longitude'],Paris_df['Neighborhood'], Paris_df['Neighborhood_Name']):
    label='{}-{}'.format(neighborhood, name)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
        ).add_to(map_Paris)
    
map_Paris

In [25]:
import folium

#We initialize the Folium map with Paris's general coordinates
Rome_lat=41.902782
Rome_long=12.496366
map_Rome = folium.Map(location=[Rome_lat, Rome_long], zoom_start=10)

#We start adding to the map the centers of the different neighborhoods listed in the dataframe we created previously
for lat, lng, neighborhood, name in zip(Rome_df['Latitude'],Rome_df['Longitude'],Rome_df['Neighborhood'], Rome_df['Neighborhood_Name']):
    label='{}-{}'.format(neighborhood, name)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
        ).add_to(map_Rome)
    
map_Rome

We can see that the Municipi in Roma are more spread out than the arrondissement in Paris. However, if we compare the totals of the populations in Paris's arrondissements and in Roma's Municipi we get a comparable figure (between 2 and 3 millions). Therefore we will assume that this is a comparable level for our analysis and we will just be careful to use different radius for our searches with the Foursquare API.

#### 3.4 Exploring the neighborhoods with the Foursquare API
Now that we have our lists of neighborhoods with their coordinates we can start analyzing and exploring them with the Foursquare API.

##### 3.4.1. Extracting the lists of venues and their category types for every neighborhood
We start by preparing our Foursquare API credentials, a function "get_category_type()" to extract the category of the venues, and another function "get_nearby_venues()", to extract the lists of venues in each neighborhood.

In [12]:
#Let's enter our Foursquare credentials and version
CLIENT_ID = 'SJPXHMOWC4S5KKV3GBLWIPSIGTY3R3QMBNOPJMU3SAY2Y1UB' # my Foursquare ID
CLIENT_SECRET = 'BC2OKDYBRT0Y2U43GFZNOJ3A4UKQH4P4CYPXYIAPNF4PVMSZ' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [13]:
#we use the get_category_type() function we defined in the labs to obtain the category of the venues
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
#We define the function to extract the venues nearby every neighborhood
LIMIT=200
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

- Rome

In [18]:
Rome_radius=10000
Rome_venues = getNearbyVenues(names=Rome_df['Neighborhood'],
                                   latitudes=Rome_df['Latitude'],
                                   longitudes=Rome_df['Longitude'],
                                  radius=Rome_radius
                                  )

Municipio Roma I
Municipio Roma II
Municipio Roma III
Municipio Roma IV
Municipio Roma V
Municipio Roma VI
Municipio Roma VII
Municipio Roma VIII
Municipio Roma IX
Municipio Roma X
Municipio Roma XI
Municipio Roma XII
Municipio Roma XIII
Municipio Roma XIV
Municipio Roma XV


In [19]:
print(Rome_venues.shape)
Rome_venues.head()

(1403, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Municipio Roma I,41.902915,12.482536,Pane e Salame,41.900604,12.481798,Sandwich Place
1,Municipio Roma I,41.902915,12.482536,La Sandwicheria,41.902901,12.483336,Sandwich Place
2,Municipio Roma I,41.902915,12.482536,Louis Vuitton,41.903849,12.478822,Boutique
3,Municipio Roma I,41.902915,12.482536,Venchi,41.900042,12.480883,Ice Cream Shop
4,Municipio Roma I,41.902915,12.482536,La Prosciutteria,41.901888,12.484467,Italian Restaurant


In [20]:
#We verify how many venues we have in each neighborhood
Rome_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Municipio Roma I,100,100,100,100,100,100
Municipio Roma II,100,100,100,100,100,100
Municipio Roma III,100,100,100,100,100,100
Municipio Roma IV,100,100,100,100,100,100
Municipio Roma IX,94,94,94,94,94,94
Municipio Roma V,100,100,100,100,100,100
Municipio Roma VI,100,100,100,100,100,100
Municipio Roma VII,100,100,100,100,100,100
Municipio Roma VIII,100,100,100,100,100,100


In [21]:
#We check how many unique categories of venues we have
print('There are {} uniques categories.'.format(len(Rome_venues['Venue Category'].unique())))

There are 135 uniques categories.


- Paris

In [22]:
Paris_radius=1000
Paris_venues = getNearbyVenues(names=Paris_df['Neighborhood'],
                                   latitudes=Paris_df['Latitude'],
                                   longitudes=Paris_df['Longitude'],
                                  radius=Paris_radius
                                  )

Paris arrondissement I
Paris arrondissement II
Paris arrondissement III
Paris arrondissement IV
Paris arrondissement V
Paris arrondissement VI
Paris arrondissement VII
Paris arrondissement VIII
Paris arrondissement IX
Paris arrondissement X
Paris arrondissement XI
Paris arrondissement XII
Paris arrondissement XIII
Paris arrondissement XIV
Paris arrondissement XV
Paris arrondissement XVI
Paris arrondissement XVII
Paris arrondissement XVIII
Paris arrondissement XIX
Paris arrondissement XX


In [23]:
print(Paris_venues.shape)
Paris_venues.head()

(1893, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Paris arrondissement I,48.834188,2.331955,Amorino,48.833763,2.329871,Ice Cream Shop
1,Paris arrondissement I,48.834188,2.331955,Poissonnerie Daguerre Marée,48.833492,2.330609,Fish Market
2,Paris arrondissement I,48.834188,2.331955,Swann et Vincent,48.833882,2.330880,Italian Restaurant
3,Paris arrondissement I,48.834188,2.331955,La Lingerie,48.837353,2.334107,Bar
4,Paris arrondissement I,48.834188,2.331955,Les Grands Voisins,48.837244,2.334895,Art Gallery


In [24]:
#We verify how many venues we have in each neighborhood
Paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Paris arrondissement I,100,100,100,100,100,100
Paris arrondissement II,100,100,100,100,100,100
Paris arrondissement III,100,100,100,100,100,100
Paris arrondissement IV,100,100,100,100,100,100
Paris arrondissement IX,100,100,100,100,100,100
Paris arrondissement V,100,100,100,100,100,100
Paris arrondissement VI,100,100,100,100,100,100
Paris arrondissement VII,100,100,100,100,100,100
Paris arrondissement VIII,100,100,100,100,100,100


In [25]:
#We check how many unique categories of venues we have
print('There are {} uniques categories.'.format(len(Paris_venues['Venue Category'].unique())))

There are 241 uniques categories.


We have reached the maximum number of 100 venues per neighborhood in most neighborhoods. We will continue our analysis with these lists.

##### 3.4.2 Analysis of the neighborhoods and the categories of venues.
Now that we have our lists of max 100 venues describing each neighborhood we will start analyzing them. First we will do a On hot encoding and group the venues category by the number of venues in each category. This will enable us to see, after some sorting, the 10 most frequent types of venues in each neighborhood.

In [26]:
#one hot encoding
Rome_onehot=pd.get_dummies(Rome_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Rome_onehot['Neighborhood']=Rome_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns=[Rome_onehot.columns[-1]]+list(Rome_onehot.columns[:-1])
Rome_onehot=Rome_onehot[fixed_columns]

Rome_onehot.head()

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,...,Temple,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Municipio Roma I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Municipio Roma I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Municipio Roma I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Municipio Roma I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Municipio Roma I,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#We group by neighborhood and average the number of venues
Rome_grouped=Rome_onehot.groupby('Neighborhood').mean().reset_index()
Rome_grouped.head(10)

,Neighborhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,...,Temple,Theater,Theme Park,Toy / Game Store,Track Stadium,Train Station,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop
0,Municipio Roma I,0.00,0.01,0.0,0.02,0.00,0.00,0.00,0.000000,0.010000,...,0.02,0.00,0.000000,0.01,0.00,0.0,0.01,0.00,0.06,0.000000
1,Municipio Roma II,0.00,0.00,0.0,0.03,0.00,0.00,0.01,0.000000,0.010000,...,0.02,0.00,0.000000,0.01,0.00,0.0,0.01,0.00,0.02,0.000000
2,Municipio Roma III,0.01,0.00,0.0,0.00,0.01,0.00,0.00,0.000000,0.010000,...,0.00,0.00,0.000000,0.01,0.00,0.0,0.01,0.00,0.00,0.010000
3,Municipio Roma IV,0.00,0.00,0.0,0.03,0.00,0.00,0.00,0.010000,0.010000,...,0.00,0.00,0.000000,0.01,0.00,0.0,0.01,0.00,0.02,0.000000
4,Municipio Roma IX,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.010638,0.031915,...,0.00,0.00,0.021277,0.00,0.00,0.0,0.00,0.00,0.00,0.010638
5,Municipio Roma V,0.00,0.00,0.0,0.02,0.00,0.00,0.00,0.000000,0.010000,...,0.02,0.00,0.000000,0.00,0.01,0.0,0.02,0.00,0.02,0.000000
6,Municipio Roma VI,0.01,0.00,0.0,0.00,0.00,0.00,0.00,0.010000,0.010000,...,0.00,0.01,0.000000,0.01,0.00,0.0,0.01,0.00,0.02,0.000000
7,Municipio Roma VII,0.00,0.00,0.0,0.01,0.00,0.00,0.00,0.000000,0.020000,...,0.00,0.00,0.000000,0.00,0.01,0.0,0.03,0.00,0.02,0.000000
8,Municipio Roma VIII,0.00,0.00,0.0,0.02,0.00,0.00,0.00,0.000000,0.030000,...,0.02,0.00,0.000000,0.00,0.01,0.0,0.02,0.00,0.01,0.000000
9,Municipio Roma X,0.00,0.00,0.0,0.00,0.01,0.01,0.00,0.010000,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.01,0.00,0.000000


In [28]:
#We create a function to to sort values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
#We create a new DataFrame to display the top 10 venues in each Neighborhood
num_top_venues=10
indicators=['st','nd','rd'] #endings for 1st, 2nd and 3rd

# create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}Most common venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))
        
# create a new dataframe
Rome_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Rome_neighborhoods_venues_sorted['Neighborhood']=Rome_grouped['Neighborhood']
for ind in np.arange(Rome_grouped.shape[0]):
    Rome_neighborhoods_venues_sorted.iloc[ind,1:]=return_most_common_venues(Rome_grouped.iloc[ind,:], num_top_venues)
    
Rome_neighborhoods_venues_sorted.head(10)

,Neighborhood,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,Municipio Roma I,Plaza,Ice Cream Shop,Historic Site,Sandwich Place,Hotel,Wine Bar,Monument / Landmark,Fountain,Church,Pizza Place
1,Municipio Roma II,Plaza,Ice Cream Shop,Hotel,Historic Site,Sandwich Place,Monument / Landmark,Pizza Place,History Museum,Park,Italian Restaurant
2,Municipio Roma III,Italian Restaurant,Pizza Place,Ice Cream Shop,Cocktail Bar,Park,Café,Clothing Store,Steakhouse,Hotel,Gym / Fitness Center
3,Municipio Roma IV,Ice Cream Shop,Italian Restaurant,Plaza,Hotel,Pizza Place,Park,Art Museum,Sandwich Place,Historic Site,Wine Bar
4,Municipio Roma IX,Pizza Place,Café,Hotel,Italian Restaurant,Ice Cream Shop,Clothing Store,Restaurant,Fast Food Restaurant,Pool,Gym / Fitness Center
5,Municipio Roma V,Ice Cream Shop,Historic Site,Plaza,Hotel,Park,Italian Restaurant,Monument / Landmark,Sandwich Place,Pizza Place,History Museum
6,Municipio Roma VI,Italian Restaurant,Pizza Place,Park,Ice Cream Shop,Hotel,Café,Gym,Pub,Restaurant,Historic Site
7,Municipio Roma VII,Pizza Place,Historic Site,Italian Restaurant,Ice Cream Shop,Park,Trattoria/Osteria,Plaza,Café,Hotel,Bakery
8,Municipio Roma VIII,Ice Cream Shop,Historic Site,Italian Restaurant,Plaza,Park,Pizza Place,Café,Hotel,Bakery,Monument / Landmark
9,Municipio Roma X,Beach,Italian Restaurant,Pizza Place,Seafood Restaurant,Ice Cream Shop,Café,Restaurant,Historic Site,Park,Chocolate Shop


We do the same for Paris's neighborhoods

In [30]:
#one hot encoding
Paris_onehot=pd.get_dummies(Paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Neighborhood']=Paris_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns=[Paris_onehot.columns[-1]]+list(Paris_onehot.columns[:-1])
Paris_onehot=Paris_onehot[fixed_columns]

#We group by neighborhood and average the number of venues
Paris_grouped=Paris_onehot.groupby('Neighborhood').mean().reset_index()

#We create a new DataFrame to display the top 10 venues in each Neighborhood
num_top_venues=10
indicators=['st','nd','rd'] #endings for 1st, 2nd and 3rd

# create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}Most common venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most common venue'.format(ind+1))
        
# create a new dataframe
Paris_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Paris_neighborhoods_venues_sorted['Neighborhood']=Paris_grouped['Neighborhood']
for ind in np.arange(Paris_grouped.shape[0]):
    Paris_neighborhoods_venues_sorted.iloc[ind,1:]=return_most_common_venues(Paris_grouped.iloc[ind,:], num_top_venues)
    
Paris_neighborhoods_venues_sorted.head(10)

,Neighborhood,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,Paris arrondissement I,French Restaurant,Hotel,Italian Restaurant,Pizza Place,Bakery,Bar,Vietnamese Restaurant,Creperie,Wine Shop,Coffee Shop
1,Paris arrondissement II,French Restaurant,Italian Restaurant,Wine Bar,Restaurant,Bistro,Coffee Shop,Hotel,Korean Restaurant,Breakfast Spot,Seafood Restaurant
2,Paris arrondissement III,French Restaurant,Hotel,Italian Restaurant,Plaza,Art Museum,Bakery,Museum,Burger Joint,Restaurant,Shopping Mall
3,Paris arrondissement IV,French Restaurant,Art Gallery,Clothing Store,Café,Hotel,Italian Restaurant,Tea Room,Cocktail Bar,Ice Cream Shop,Pastry Shop
4,Paris arrondissement IX,French Restaurant,Hotel,Wine Bar,Italian Restaurant,Pizza Place,Bakery,Cocktail Bar,Bar,Restaurant,Gourmet Shop
5,Paris arrondissement V,French Restaurant,Bakery,Plaza,Ice Cream Shop,Coffee Shop,Bistro,Science Museum,Hotel,Pastry Shop,Park
6,Paris arrondissement VI,French Restaurant,Hotel,Wine Bar,Art Gallery,Ice Cream Shop,Coffee Shop,Historic Site,Bakery,Café,Burger Joint
7,Paris arrondissement VII,Hotel,French Restaurant,Boutique,Pastry Shop,Chocolate Shop,Gourmet Shop,Plaza,Men's Store,Hotel Bar,Dessert Shop
8,Paris arrondissement VIII,Hotel,French Restaurant,Art Gallery,Pastry Shop,Italian Restaurant,Japanese Restaurant,Theater,Bakery,Boutique,Spa
9,Paris arrondissement X,French Restaurant,Coffee Shop,Pizza Place,Japanese Restaurant,Café,Korean Restaurant,Cocktail Bar,Italian Restaurant,Seafood Restaurant,Indian Restaurant


##### 3.4.3 Clustering
Now that we have a description of the most common types of venues in each neighborhood we can start the clustering. First we will merge both lists as the goal is to see if any clusters can be formed between neighborhoods in Paris and in Rome.

In [31]:
Rome_and_Paris_grouped=pd.concat([Rome_grouped, Paris_grouped], ignore_index=True).fillna(0)
Rome_and_Paris_neighborhoods_venues_sorted=pd.concat([Rome_neighborhoods_venues_sorted, Paris_neighborhoods_venues_sorted], ignore_index=True)
Rome_and_Paris_data=pd.concat([Rome_df, Paris_df], ignore_index=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [32]:
from sklearn.cluster import KMeans
# set number of clusters
k_clusters=6
Rome_and_Paris_grouped_clustering=Rome_and_Paris_grouped.drop('Neighborhood',1)

# run k-means clustering
Kmeans=KMeans(n_clusters=k_clusters, random_state=0).fit(Rome_and_Paris_grouped_clustering)

# add clustering labels
Rome_and_Paris_neighborhoods_venues_sorted.insert(0, 'Cluster labels', Kmeans.labels_)
Rome_and_Paris_merged=Rome_and_Paris_data

# merge Rome_and_Paris_merged with Rome_and_Paris_data to add latitude/longitude for each neighborhood
Rome_and_Paris_merged=Rome_and_Paris_merged.join(Rome_and_Paris_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

##### 3.4.4 Clusters analysis
Finally, let's display and observe the clusters formed. It is interesting to see if the KMeans Clustering method has formed any clusters grouping neighborhoods belong to Rome and Paris.

- Cluster 0:

In [33]:
Cluster_0=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==0,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_0

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
0,Municipio Roma I,Centro Storico,Plaza,Ice Cream Shop,Historic Site,Sandwich Place,Hotel,Wine Bar,Monument / Landmark,Fountain,Church,Pizza Place
1,Municipio Roma II,Parioli/Nomentano,Plaza,Ice Cream Shop,Hotel,Historic Site,Sandwich Place,Monument / Landmark,Pizza Place,History Museum,Park,Italian Restaurant
3,Municipio Roma IV,Tiburtina,Ice Cream Shop,Italian Restaurant,Plaza,Hotel,Pizza Place,Park,Art Museum,Sandwich Place,Historic Site,Wine Bar
4,Municipio Roma V,Prenestino/Centocelle,Ice Cream Shop,Historic Site,Plaza,Hotel,Park,Italian Restaurant,Monument / Landmark,Sandwich Place,Pizza Place,History Museum
7,Municipio Roma VIII,Appia Antica,Ice Cream Shop,Historic Site,Italian Restaurant,Plaza,Park,Pizza Place,Café,Hotel,Bakery,Monument / Landmark
11,Municipio Roma XII,Monte Verde,Italian Restaurant,Ice Cream Shop,Pizza Place,Sandwich Place,Hotel,Plaza,Church,Café,Scenic Lookout,Bakery


- Cluster 1:

In [34]:
Cluster_1=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==1,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_1

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
16,Paris arrondissement II,Bourse,French Restaurant,Italian Restaurant,Wine Bar,Restaurant,Bistro,Coffee Shop,Hotel,Korean Restaurant,Breakfast Spot,Seafood Restaurant
18,Paris arrondissement IV,Hôtel-de-Ville,French Restaurant,Art Gallery,Clothing Store,Café,Hotel,Italian Restaurant,Tea Room,Cocktail Bar,Ice Cream Shop,Pastry Shop
19,Paris arrondissement V,Panthéon,French Restaurant,Bakery,Plaza,Ice Cream Shop,Coffee Shop,Bistro,Science Museum,Hotel,Pastry Shop,Park
20,Paris arrondissement VI,Luxembourg,French Restaurant,Hotel,Wine Bar,Art Gallery,Ice Cream Shop,Coffee Shop,Historic Site,Bakery,Café,Burger Joint
21,Paris arrondissement VII,Palais-Bourbon,Hotel,French Restaurant,Boutique,Pastry Shop,Chocolate Shop,Gourmet Shop,Plaza,Men's Store,Hotel Bar,Dessert Shop
23,Paris arrondissement IX,Opéra,French Restaurant,Hotel,Wine Bar,Italian Restaurant,Pizza Place,Bakery,Cocktail Bar,Bar,Restaurant,Gourmet Shop
24,Paris arrondissement X,Entrepôt,French Restaurant,Coffee Shop,Pizza Place,Japanese Restaurant,Café,Korean Restaurant,Cocktail Bar,Italian Restaurant,Seafood Restaurant,Indian Restaurant
25,Paris arrondissement XI,Popincourt,French Restaurant,Pizza Place,Hotel,Cocktail Bar,Clothing Store,Italian Restaurant,Plaza,Art Gallery,Tapas Restaurant,Coffee Shop
29,Paris arrondissement XV,Vaugirard,French Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Sandwich Place,Beer Garden,Bistro,Bar,Pizza Place,Supermarket
32,Paris arrondissement XVIII,Butte-Montmartre,French Restaurant,Bar,Bistro,Pizza Place,Italian Restaurant,Café,Restaurant,Plaza,Deli / Bodega,Bakery


- Cluster 2:

In [35]:
Cluster_5=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==2,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_5

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
26,Paris arrondissement XII,Reuilly,Bus Stop,Hotel,Supermarket,Park,Restaurant,Middle Eastern Restaurant,Metro Station,Shipping Store,Flea Market,Music Venue


- Cluster 3:

In [36]:
Cluster_3=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==3,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_3

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
2,Municipio Roma III,Monte Sacro,Italian Restaurant,Pizza Place,Ice Cream Shop,Cocktail Bar,Park,Café,Clothing Store,Steakhouse,Hotel,Gym / Fitness Center
5,Municipio Roma VI,Roma delle Torri,Italian Restaurant,Pizza Place,Park,Ice Cream Shop,Hotel,Café,Gym,Pub,Restaurant,Historic Site
6,Municipio Roma VII,Appio-Latino/Tuscolana/Cinecittà,Pizza Place,Historic Site,Italian Restaurant,Ice Cream Shop,Park,Trattoria/Osteria,Plaza,Café,Hotel,Bakery
8,Municipio Roma IX,Eur,Pizza Place,Café,Hotel,Italian Restaurant,Ice Cream Shop,Clothing Store,Restaurant,Fast Food Restaurant,Pool,Gym / Fitness Center
9,Municipio Roma X,Ostia/Acilia,Beach,Italian Restaurant,Pizza Place,Seafood Restaurant,Ice Cream Shop,Café,Restaurant,Historic Site,Park,Chocolate Shop
10,Municipio Roma XI,Arvalia/Portuense,Pizza Place,Ice Cream Shop,Italian Restaurant,Café,Hotel,Park,Gym / Fitness Center,Athletics & Sports,Bakery,Resort
12,Municipio Roma XIII,Aurelia,Italian Restaurant,Hotel,Café,Pizza Place,Restaurant,Ice Cream Shop,Gourmet Shop,Gym / Fitness Center,Pub,Supermarket
13,Municipio Roma XIV,Monte Mario,Italian Restaurant,Café,Pub,Hotel,Gym / Fitness Center,Asian Restaurant,Toy / Game Store,Pizza Place,Supermarket,Chinese Restaurant
14,Municipio Roma XV,Cassia/Flaminia,Italian Restaurant,Pizza Place,Restaurant,Supermarket,Café,Ice Cream Shop,Pub,Stadium,Clothing Store,Hotel


- Cluster 4:

In [37]:
Cluster_4=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==4,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_4

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
15,Paris arrondissement I,Louvre,French Restaurant,Hotel,Italian Restaurant,Pizza Place,Bakery,Bar,Vietnamese Restaurant,Creperie,Wine Shop,Coffee Shop
17,Paris arrondissement III,Temple,French Restaurant,Hotel,Italian Restaurant,Plaza,Art Museum,Bakery,Museum,Burger Joint,Restaurant,Shopping Mall
22,Paris arrondissement VIII,Élysée,Hotel,French Restaurant,Art Gallery,Pastry Shop,Italian Restaurant,Japanese Restaurant,Theater,Bakery,Boutique,Spa
28,Paris arrondissement XIV,Observatoire,French Restaurant,Hotel,Italian Restaurant,Bar,Bakery,Sushi Restaurant,Pizza Place,Japanese Restaurant,Bistro,Vietnamese Restaurant
30,Paris arrondissement XVI,Passy,French Restaurant,Hotel,Italian Restaurant,Seafood Restaurant,Electronics Store,Moroccan Restaurant,Japanese Restaurant,Bakery,Steakhouse,Bistro
31,Paris arrondissement XVII,Batignolles-Monceau,French Restaurant,Italian Restaurant,Hotel,Bakery,Restaurant,Park,Pastry Shop,Creperie,Bistro,Bar


- Cluster 5:

In [38]:
Cluster_5=Rome_and_Paris_merged.loc[Rome_and_Paris_merged['Cluster labels']==5,Rome_and_Paris_merged.columns[[0]+[1]+list(range(5,Rome_and_Paris_merged.shape[1]))]]
Cluster_5

,Neighborhood,Neighborhood_Name,1stMost common venue,2ndMost common venue,3rdMost common venue,4th Most common venue,5th Most common venue,6th Most common venue,7th Most common venue,8th Most common venue,9th Most common venue,10th Most common venue
27,Paris arrondissement XIII,Gobelins,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,Italian Restaurant,Bakery,French Restaurant,Supermarket,Cambodian Restaurant,Cantonese Restaurant


#### 3.5 Food scene diversity
The second question we posed was to estimate the food scene diversity in both capitals. To do so, we will filter out dataframes containing the venues categories to only keep the categories corresponding to restaurants types. We will calculate in each city:
- the total number of restaurants,  
- the number of different types of restaurants,
- the number of types of French Restaurants in Paris and of Italian Restaurants in Rome
- the total number of restaurants of these types in Paris and Rome
- the number of types of restaurant outside of French Restaurants in Paris and of Italian Restaurants in Rome
- the total number of restaurants of these types in Paris and Rome.

##### Rome:

In [73]:
Rome_grouped_total=Rome_onehot.groupby('Neighborhood').sum().reset_index()
restaurants_types_list=['American Restaurant',
 'Argentinian Restaurant','Asian Restaurant','Bistro','Brazilian Restaurant','Buffet','Chinese Restaurant','Cuban Restaurant','Diner',
 'Ethiopian Restaurant','Fast Food Restaurant','Burger Joint','Food Truck','French Restaurant','Greek Restaurant','Italian Restaurant',
 'Japanese Restaurant','Kebab Restaurant','Mediterranean Restaurant','Pizza Place','Restaurant','Roman Restaurant','Steakhouse','Seafood Restaurant','Spanish Restaurant','Sushi Restaurant','Trattoria/Osteria',
 'Vegetarian / Vegan Restaurant']
cols = [col for col in Rome_grouped_total.columns if col in restaurants_types_list]
Rome_grouped_total = Rome_grouped_total[cols]

In [76]:
Rome_total_restaurants=pd.DataFrame(Rome_grouped_total.sum(axis = 0, skipna = True))
Rome_total_restaurants=Rome_total_restaurants.reset_index()
Rome_total_restaurants=Rome_total_restaurants.rename(columns={"index": "Restaurant type", 0: "Rome Total"})
Rome_total_restaurants

,Restaurant type,Rome Total
0,American Restaurant,3
1,Argentinian Restaurant,1
2,Asian Restaurant,6
3,Bistro,7
4,Brazilian Restaurant,2
5,Buffet,2
6,Burger Joint,3
7,Chinese Restaurant,6
8,Cuban Restaurant,1
9,Diner,9


In [84]:
print('There are {} different types of restaurants in Rome'.format(Rome_total_restaurants.shape[0]))

There are 28 different types of restaurants in Rome


In [86]:
Italian_restaurant_types=['Italian Restaurant','Pizza Place','Roman Restaurant','Roman Restaurant','Trattoria/Osteria']
Italian_restaurant_total=Rome_total_restaurants[Rome_total_restaurants['Restaurant type'].isin(Italian_restaurant_types)]
Italian_restaurant_total

,Restaurant type,Rome Total
15,Italian Restaurant,132
19,Pizza Place,102
21,Roman Restaurant,3
26,Trattoria/Osteria,14


In [87]:
Italian_restaurant_total.sum()

Restaurant type    Italian RestaurantPizza PlaceRoman RestaurantT...
Rome Total                                                       251
dtype: object

##### Paris:

In [75]:
Paris_grouped_total=Paris_onehot.groupby('Neighborhood').sum().reset_index()
restaurants_types_list2=['African Restaurant','Alsatian Restaurant','American Restaurant','Arepa Restaurant','Argentinian Restaurant','Asian Restaurant','Auvergne Restaurant','Basque Restaurant','Belgian Restaurant',
 'Bistro','Brasserie','Breton Restaurant','Burger Joint','Cajun / Creole Restaurant','Cambodian Restaurant','Cantonese Restaurant',
 'Caribbean Restaurant','Chinese Restaurant','Corsican Restaurant','Creperie','Eastern European Restaurant','Empanada Restaurant','English Restaurant','Ethiopian Restaurant','Falafel Restaurant','Fast Food Restaurant','Food Court',
 'Food Truck','French Restaurant','Gluten-free Restaurant','Greek Restaurant','Halal Restaurant','Hot Dog Joint','Indian Restaurant','Israeli Restaurant','Italian Restaurant','Japanese Restaurant','Jewish Restaurant','Korean Restaurant',
 'Latin American Restaurant','Lebanese Restaurant','Mediterranean Restaurant','Mexican Restaurant','Middle Eastern Restaurant','Modern European Restaurant',
 'Molecular Gastronomy Restaurant','Moroccan Restaurant','Okonomiyaki Restaurant','Peruvian Restaurant','Pizza Place','Portuguese Restaurant','Provençal Restaurant','Restaurant','Russian Restaurant','Salad Place','Savoyard Restaurant',
 'Scandinavian Restaurant','Seafood Restaurant','Shanxi Restaurant','Soba Restaurant','Southwestern French Restaurant','Steakhouse','Sushi Restaurant','Szechuan Restaurant',
 'Taco Place','Tapas Restaurant','Thai Restaurant','Trattoria/Osteria','Turkish Restaurant','Vegetarian / Vegan Restaurant','Venezuelan Restaurant','Vietnamese Restaurant',]
cols2 = [col for col in Paris_grouped_total.columns if col in restaurants_types_list2]
Paris_grouped_total = Paris_grouped_total[cols2]

Paris_total_restaurants=pd.DataFrame(Paris_grouped_total.sum(axis = 0, skipna = True))
Paris_total_restaurants=Paris_total_restaurants.reset_index()
Paris_total_restaurants=Paris_total_restaurants.rename(columns={"index": "Restaurant type", 0: "Paris Total"})
Paris_total_restaurants


,Restaurant type,Paris Total
0,African Restaurant,5
1,Alsatian Restaurant,1
2,American Restaurant,3
3,Arepa Restaurant,1
4,Argentinian Restaurant,5
...,...,...
67,Trattoria/Osteria,1
68,Turkish Restaurant,1
69,Vegetarian / Vegan Restaurant,16
70,Venezuelan Restaurant,1


In [83]:
print('There are {} different types of restaurants in Paris'.format(Paris_total_restaurants.shape[0]))

There are 72 different types of restaurants in Paris


In [89]:
French_restaurant_types=['Alsatian Restaurant','Auvergne Restaurant','Basque Restaurant','Brasserie','Breton Restaurant','Corsican Restaurant','Creperie','French Restaurant','Provençal Restaurant','Savoyard Restaurant','Southwestern French Restaurant']
Paris_restaurant_total=Paris_total_restaurants[Paris_total_restaurants['Restaurant type'].isin(French_restaurant_types)]
Paris_restaurant_total

,Restaurant type,Paris Total
1,Alsatian Restaurant,1
6,Auvergne Restaurant,3
7,Basque Restaurant,1
10,Brasserie,4
11,Breton Restaurant,1
18,Corsican Restaurant,3
19,Creperie,14
28,French Restaurant,276
51,Provençal Restaurant,2
55,Savoyard Restaurant,1


In [90]:
Paris_restaurant_total.sum()

Restaurant type    Alsatian RestaurantAuvergne RestaurantBasque R...
Paris Total                                                      309
dtype: object

In [81]:
Rome_and_Paris_restaurants_total=pd.concat([Paris_total_restaurants,Rome_total_restaurants],ignore_index=True).fillna(0)
columns=['Restaurant type','Paris Total','Rome Total']
Rome_and_Paris_restaurants_total[columns]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Restaurant type,Paris Total,Rome Total
0,African Restaurant,5.0,0.0
1,Alsatian Restaurant,1.0,0.0
2,American Restaurant,3.0,0.0
3,Arepa Restaurant,1.0,0.0
4,Argentinian Restaurant,5.0,0.0
...,...,...,...
95,Spanish Restaurant,0.0,1.0
96,Steakhouse,0.0,7.0
97,Sushi Restaurant,0.0,5.0
98,Trattoria/Osteria,0.0,14.0


In [82]:
Rome_and_Paris_restaurants_total.sum(axis = 0, skipna = True) 

Paris Total                                                      863
Restaurant type    African RestaurantAlsatian RestaurantAmerican ...
Rome Total                                                       379
dtype: object

#### 3.6 Restaurants ratings
Now we want to answer the third problem we posed for our study which is to decide on which city has the highest ratings for their restaurants. To do so, we first need to gather the lists of restaurants in both cities, with their Foursquare unique ID. Once we have the lists of restaurants, we will run another series of API calls to obtain the ratings of the restaurants listed.  
*Note:* Since we are limited in the number of API calls we can perform every day, we will reduce the perimeter of our study. We will only consider restaurants in the 5 most center neighborhoods of Rome, meaning restaurants in the neighborhoods called Municipio Roma I to V. Likewise, we will reduce the perimeter of the analysis in Paris to the 5 most center arrondissements (being Paris arrondissement 1 to 5). Finally, another limitation we will make to our study is to only compare French Restaurants' ratings in Paris to Italian Restaurants' ratings in Rome.

##### 3.6.1 Scope definition

In [15]:
#We define the function to extract the IDs of the venues nearby every neighborhood
LIMIT=100
def getIDs(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues_IDs = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues_IDs.columns = ['Neighborhood',
                  'Venue', 
                  'Venue ID',
                  'Venue Category']
    
    return(nearby_venues_IDs)

##### Paris:

In [130]:
#We limit the scope to the 5 most central arrondissement in Paris to limit the number of API calls to perform
Paris_scope=['Paris arrondissement I','Paris arrondissement II','Paris arrondissement III','Paris arrondissement IV','Paris arrondissement V']
Paris_df2=Paris_df[Paris_df['Neighborhood'].isin(Paris_scope)].reset_index(drop=True)
Paris_df2

,Neighborhood,Neighborhood_Name,Latitude,Longitude
0,Paris arrondissement I,Louvre,48.834188,2.331955
1,Paris arrondissement II,Bourse,48.867053,2.363482
2,Paris arrondissement III,Temple,48.863583,2.287129
3,Paris arrondissement IV,Hôtel-de-Ville,48.857323,2.362473
4,Paris arrondissement V,Panthéon,48.845419,2.352582


In [131]:
Paris_IDs = getIDs(names=Paris_df2['Neighborhood'],
                            latitudes=Paris_df2['Latitude'],
                            longitudes=Paris_df2['Longitude'],
                            radius=Paris_radius
                            )
#We also limit the scope to French restaurants only to limit the number of API calls to perform
Paris_IDs=Paris_IDs[Paris_IDs['Venue Category']=='French Restaurant'].reset_index(drop=True)
print('We have {} restaurants in the scope in Paris'.format(Paris_IDs.shape[0]))
Paris_IDs.head(10)

Paris arrondissement I
Paris arrondissement II
Paris arrondissement III
Paris arrondissement IV
Paris arrondissement V
We have 78 restaurants in the scope in French


,Neighborhood,Venue,Venue ID,Venue Category
0,Paris arrondissement I,À Mi-Chemin,4b2b4a65f964a52099b524e3,French Restaurant
1,Paris arrondissement I,Le Cornichon,4cbde759dd41a35de4cefba0,French Restaurant
2,Paris arrondissement I,Aux Plumes,580f9ce938faaa7ba63399c0,French Restaurant
3,Paris arrondissement I,Au Bistrot d'à Côté,4c3e0de2b0379521eb089ba7,French Restaurant
4,Paris arrondissement I,La Maison Courtine,4e31cbf0d22dc6a915312b59,French Restaurant
5,Paris arrondissement I,Le Cantine du Troquet,54d1d78e498e60ecf1b0476b,French Restaurant
6,Paris arrondissement I,Kigawa,4d9f5e458e8f224b4690a256,French Restaurant
7,Paris arrondissement I,Le Severo,4ba26c57f964a52093f737e3,French Restaurant
8,Paris arrondissement I,La Closerie des Lilas,4adcda05f964a520733221e3,French Restaurant
9,Paris arrondissement I,Le Petit Baigneur,4b828edaf964a5203cd830e3,French Restaurant


##### Rome:

In [16]:
#We limit the scope to the 5 most central Muncipi in Rome to limit the number of API calls to perform
Rome_scope=['Municipio Roma I','Municipio Roma II','Municipio Roma III','Municipio Roma IV','Municipio Roma V']
Rome_df2=Rome_df[Rome_df['Neighborhood'].isin(Rome_scope)].reset_index(drop=True)
Rome_df2

,Neighborhood,Neighborhood_Name,Latitude,Longitude
0,Municipio Roma I,Centro Storico,41.902915,12.482536
1,Municipio Roma II,Parioli/Nomentano,41.917486,12.499319
2,Municipio Roma III,Monte Sacro,42.001921,12.555902
3,Municipio Roma IV,Tiburtina,41.932972,12.597107
4,Municipio Roma V,Prenestino/Centocelle,41.894312,12.573932


In [18]:
Rome_radius=10000
Rome_IDs = getIDs(names=Rome_df2['Neighborhood'],
                            latitudes=Rome_df2['Latitude'],
                            longitudes=Rome_df2['Longitude'],
                            radius=Rome_radius
                            )
Rome_IDs=Rome_IDs[Rome_IDs['Venue Category']=='Italian Restaurant'].reset_index(drop=True)
print('We have {} restaurants in the scope in Rome'.format(Rome_IDs.shape[0]))
Rome_IDs.head(10)

Municipio Roma I
Municipio Roma II
Municipio Roma III
Municipio Roma IV
Municipio Roma V
We have 41 restaurants in the scope in Rome


,Neighborhood,Venue,Venue ID,Venue Category
0,Municipio Roma I,La Prosciutteria,52fa76ba498ebc45669803ce,Italian Restaurant
1,Municipio Roma I,Cantina e Cucina,4d84a28b8de9721e0fa23451,Italian Restaurant
2,Municipio Roma II,Al Forno della Soffitta,4adcdacbf964a520015521e3,Italian Restaurant
3,Municipio Roma II,Fiaschetteria Marini,4da895301e72d9bb474978a5,Italian Restaurant
4,Municipio Roma II,Culinaria,5589b12c498efce8c10fd4d2,Italian Restaurant
5,Municipio Roma II,La Prosciutteria,52fa76ba498ebc45669803ce,Italian Restaurant
6,Municipio Roma III,Antica Hostaria dei Ghiottoni,4d0d497fbdbfa35d90c96872,Italian Restaurant
7,Municipio Roma III,Ristorante Da Garibaldi,4e0b7499483bb906b6b241fc,Italian Restaurant
8,Municipio Roma III,Che Te Ne Sa,4c2f8bdd7cc0c9b61eeceb9a,Italian Restaurant
9,Municipio Roma III,Mama's Restaurant,4b607204f964a52066e629e3,Italian Restaurant


##### 3.6.2 Ratings extraction
##### Paris:

In [20]:
from pandas.io.json import json_normalize
#We define the function to extract the ratings of every venue
LIMIT=100
def getRating(ids):
    
    ids_list=[]
    ratings_list=[]
    
    for venue_id in ids:
        ids_list.append(venue_id)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
            
        # make the GET request
        result = requests.get(url).json()
        
        # return only the rating for each nearby venue if it is available
        try: 
            ratings_list.append(result['response']['venue']['rating'])
        except:
            ratings_list.append('No rating')
    
    dict={'Id':ids_list,'Rating':ratings_list}
    venues_ratings = pd.DataFrame(dict)
    
    return(venues_ratings)

In [100]:
#We run our function on the list of ratings of the restaurants in the scope in Paris
Paris_ratings = getRating(ids=Paris_IDs['Venue ID'])

In [103]:
Paris_ratings.head(20)

,Id,Rating
0,4b2b4a65f964a52099b524e3,8.1
1,4cbde759dd41a35de4cefba0,8.4
2,580f9ce938faaa7ba63399c0,8.1
3,4c3e0de2b0379521eb089ba7,7.8
4,4e31cbf0d22dc6a915312b59,8.4
5,54d1d78e498e60ecf1b0476b,8.4
6,4d9f5e458e8f224b4690a256,8.1
7,4ba26c57f964a52093f737e3,8.2
8,4adcda05f964a520733221e3,8.6
9,4b828edaf964a5203cd830e3,8.2


In [125]:
#In order to be able to calculate an average we again reduce the scope to the restaurants having a rating.
Paris_ratings_available=Paris_ratings[Paris_ratings['Rating']!="No rating"]

In [126]:
#We calculate the average rating on the remaining scope and print out the result
Paris_ratings_available.Rating = Paris_ratings_available.Rating.astype(float).fillna(0.0)
print('The average rating of Paris restaurants over the scope of {} restaurants is {}'.format(Paris_ratings_available.shape[0],Paris_ratings_available['Rating'].mean()))

The average rating of Paris restaurants over the scope of 47 restaurants is 8.287234042553191


##### Rome:
We now do the same on the list of restaurants in Rome.

In [21]:
#We run our function on the list of ratings of the restaurants in the scope in Paris
Rome_ratings = getRating(ids=Rome_IDs['Venue ID'])
Rome_ratings.head(20)

,Id,Rating
0,52fa76ba498ebc45669803ce,9.1
1,4d84a28b8de9721e0fa23451,9.0
2,4adcdacbf964a520015521e3,8.8
3,4da895301e72d9bb474978a5,8.7
4,5589b12c498efce8c10fd4d2,8.8
5,52fa76ba498ebc45669803ce,9.1
6,4d0d497fbdbfa35d90c96872,8.9
7,4e0b7499483bb906b6b241fc,8.1
8,4c2f8bdd7cc0c9b61eeceb9a,7.8
9,4b607204f964a52066e629e3,7.7


In [22]:
#In order to be able to calculate an average we again reduce the scope to the restaurants having a rating.
Rome_ratings_available=Rome_ratings[Rome_ratings['Rating']!="No rating"]

#We calculate the average rating on the remaining scope and print out the result
Rome_ratings_available.Rating = Rome_ratings_available.Rating.astype(float).fillna(0.0)
print('The average rating of Rome restaurants over the scope of {} restaurants is {}'.format(Rome_ratings_available.shape[0],Rome_ratings_available['Rating'].mean()))

The average rating of Rome restaurants over the scope of 41 restaurants is 8.207317073170733


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## 4. Results

#### 4.1 Similarity between Paris's and Rome's neighborhoods
The results of the KMeans clustering analysis carried out in section 3.4 show that with 6 clusters created, the neighborhoods in Rome and Paris are too different to be classified together. Indeed, the clusters created only contain neighborhoods of one city, but never of both.

#### 4.2 Food scene diversity
The results of the food scene diversity analysis are summarized in the table below:

|                                                                        | Rome          | Paris        | 
| -----------------------------------------------------------------------|:-------------:| :-----------:|
| **Total number of restaurants**                                        | 379           | 863          |
| **Number of restaurants types**                                        | 28            | 72           |
| **Number of national restaurants types**                               | 4             | 11           |
| **Total number of restaurants of national restaurants types**          | 251           | 309          |
| **Number of restaurants types outside of national restaurants types**  | 24            | 61           |
| **Total number of restaurants outside of national restaurants types**  | 128           | 554          |

This shows that even when taking out the different types of French regional restaurant types into the calculations (11 types representing 309 restaurants)and doing the same with Italian national restaurant types (4 types representing 251 restaurants) there are still 61 types of restaurants in Paris (representing 554 restaurants) versus 24 types in Rome (representing 128 restaurants). We can infer that Paris has the most diverse food scene.

#### 4.3 Restaurants ratings in Paris and Rome
The following table summarizes the results of the analysis of the ratings, with a reminder of the scope used and the calculation of the average rating over that scope.

|                                                            | Rome     | Paris    | 
| -----------------------------------------------------------|:--------:| :-------:|
| **Number of restaurants in the scope**                     | 41       | 47       |
| **Average rating of the restaurants withing that scope**   | 8.21     | 8.29     |

As we can see, the results are very close over scopes that are in fact quite similar in size and can therefore be compared. The ratings seems to be slightly higher in Paris than in Rome but it would be interesting to know if the trend remains if we extend the scope by including more neighborhoods and more types of restaurants.

## 5. Discussion
#### 5.1 Clustering analysis
The results of the clustering analysis are however partial. Indeed, it would be needed to observe the results of the KMeans clustering approach when trying with different numbers of clusters (in the present analysis, a number of 6 clusters has been used). It is however unlikely that with higher number of clusters we would start seeing clusters containing neighborhoods of both cities, as the level of detail gets higher. Therefore, it would be interesting to do such a story but on focusing on only certain parts of both cities, and breaking them down into more precise subdivisions that arrondissements and municipi. For example, by breaking down Municipio I in Roma into quartieri, an existing subdivision, we could potentially focus on some more specific areas of the very center of Rome, such as the historical center. We could then compare them to the same type of areas in Paris using the subdivision into 'quartiers'  and comparing to Paris's historical center as well to see how both areas differ. It could be interesting also to do a similar analysis on the business centers of both cities.

#### 5.2 Food scene diversity
The analysis on the types of restaurants appearing in the categories defined in Foursquare seem to show a way more diverse food scene in Paris than in Rome. This can be observed through the number of different types of restaurants, including or not regional and national types. This does not necessarily come as a surprise as while both cities attract very important numbers of international tourists as mentioned in the Introduction section, Paris is known to be a more multi-cultural city than Rome, which would explain the higher diversity in restaurants created in Paris. However, it is important to note that a limit to the conclusion of this study is that the system of categories defined on Foursquare is not necessarily homogeneous and some categories might in fact intersect or on the contrary should be broken down to more details. For example, in the list of categories of restaurants found in Paris we can see examples of French regional cuisines such as 'Alsatian', 'Savoyard', 'Basque' or 'Southwestern', while in the list of categories of Rome's restaurants there are only 4 categories referring to Italian cuisine which are 'Italian restaurant', 'Pizza Place', 'Roman restaurant' and 'Trattoria/Osteria'. We could have imagined more categories in Rome such as 'Sicilian Restaurant', and other regional cuisines. The point of the comment here is that food scene diversity remains something that is difficult to analyze analytically if not defined and measured precisely and in the case of the present study, the definition of "venue categories" in Foursquare does not seem to be precise and homogeneous enough to do so.

#### 5.3 Rome VS Paris restaurants
The results of the analysis are also very partial. Due to the daily number of API calls constraints we were not able to expand the scope of the study, therefore resulting on a very limited scope, both geographically and in terms of restaurants included. It would be interesting to expand the scope of the study to see if the trend observed remains.  
Furthermore, Foursquare users' ratings is a very subjective and partial measure of the actual quality of a restaurant. As mentioned in the Introduction section, there might not be the same userbase in both cities and the ratings given depend on very diverse factors. Although it is not possible to improve the quality of the results and conclusions here with the data used, it is important to keep these limitations in mind when analyzing the results.

## 6. Conclusion
This analysis shows that the different neighborhoods in Paris and Rome are in fact quite dissimilar although inside of each city, clusters can be formed. The food scene seems to be more diverse in Paris than it is in Rome, which makes sense since Paris is known to be a more multi-cultural city. Finally, while very limited, the results of the analysis on the restaurants' ratings seem to show a trend according to which Paris's restaurants have slightly higher ratings than Rome, providing French people (at least Parisians) something to be proud of over their Roman counterparts.